<a href="https://colab.research.google.com/github/geniusserg/Sportik/blob/master/EventsCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import libs and load functions

In [ ]:
import numpy as np
import pandas as pd
from itertools import chain
import requests
import json
import re
import os
from tqdm import tqdm

queries_for_ids = {
    "athletes_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes",
    "games_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events?limit=100",
    "weeks_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks"
}

def unwrap_reference(obj, unwrap_elements=False):
  url = ""
  if (isinstance(obj, dict)):
    url = obj["$ref"]
  elif (isinstance(obj, str)):
    url = obj
  else:
    raise Exception("unsupported input type")
  obj = requests.get(url).json()
  result_objs = []
  if 'pageIndex' in obj.keys():
    if (obj["pageCount"] == 0):
      return []
    if (obj["pageCount"] == 1):
      if (unwrap_elements):
        for i, _ in enumerate(obj["items"]):
          obj["items"][i] = requests.get(obj["items"][i]["$ref"]).json()
      return obj["items"]
    for i in range(obj['pageCount']):
      page_objs = requests.get(url, params={"page": i + 1, "limit": 1000}).json()['items']
      if (unwrap_elements):
        for i, _ in enumerate(page_objs):
          page_objs[i] = requests.get(page_objs[i]["$ref"]).json()
      result_objs.extend(page_objs)
  else:
    return obj
  return result_objs

def extract_ids_from_url(url):
  matches = re.finditer(r"([^\/?]+)", url)
  ids = {}
  current_id = ""
  for matchNum, match in enumerate(matches):
    if (matchNum < 6 or "=" in match.group()):
      continue
    try:
      ids[current_id+"_id"] = int(match.group())
    except:
      current_id = match.group()
  return ids

def statistics_unwrap(obj):
  categories = {}
  for category in obj:
    stats = {}
    cat_name = category["name"]
    for stat in category["stats"]:
      stat = {"name": stat["name"], "value": stat["value"]}
      stats[stat["name"]] = stat
    del category["stats"]
    category.update(stats)
    categories[cat_name] = category
  return categories

## Function for games

In [ ]:
def load_game(event):
  base = event["competitions"][0]["$ref"]
  game = requests.get(base).json() # object with game info

  # teams
  teams = [unwrap_reference(competitor) for competitor in game["competitors"]]

  # team
  for team in teams:
    # do not unwrap team section
    if ("score" in team):
      team["score"] = unwrap_reference(team["score"]) # overall team score in the game
    if ("linescores" in team):
      team["linescores"] = unwrap_reference(team["linescores"])  # scores by quarters
    if ("roster" in team):
      team["roster"] = unwrap_reference(team["roster"])["entries"] # list of team with players statistics
        #statistics for players
      for player in team["roster"]:
        if ("statistics" in player):
          player["statistics"] = unwrap_reference(player["statistics"])["splits"]["categories"]
          player["statistics"] = statistics_unwrap(player["statistics"])

    if ("statistics" in team):
      team["statistics"] = unwrap_reference(team["statistics"])["splits"]["categories"]  # team stats
    if ("record" in team):
      team["record"] = unwrap_reference(team["record"]) # I dont know what is it, some statistics


  game["competitors"] = teams

  # other game stats
  if ("details" in game):
    game["details"] = unwrap_reference(game["details"]) # game moments
  if ("leaders" in game):
    game["leaders"] = unwrap_reference(game["leaders"]) # game leaders by categroies
  if ("officials" in game):
    game["officials"] = unwrap_reference(game["officials"]) # ???
  if ("drives" in game):
    game["drives"] = unwrap_reference(game["drives"]) # some play events i dont know

  # delete trash
  bad_sections = set(["situation","broadcasts",\
                "status", "links",\
                "type", "necessary", \
                "timeValid", \
                "neutralSite", \
                "divisionCompetition", \
                "conferenceCompetition",\
                "onWatchESPN", "recent", "ranks"])

  for section in list(game.keys()):
    if (section in bad_sections):
      del game[section]
    if (section.find("Available") != -1):
      del game[section]
    if (section.find("Source") != -1):
      del game[section]
  if ("venue" in game):
    game["venue"].pop("images", None)

  #save json
  game_id = game["id"]
  json.dump(game, open(f"games/game_{game_id}.json", "w"))
  return game
# game = load_game(requests.get("http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/271014026?lang=en&region=us").json())

In [ ]:
game

In [ ]:
game["competitors"][0]["roster"][0]["statistics"]

{'general': {'name': 'general',
  'displayName': 'General',
  'shortDisplayName': 'General',
  'abbreviation': 'gen',
  'summary': '',
  'fumblesForced': {'name': 'fumblesForced', 'value': 0.0},
  'fumblesRecovered': {'name': 'fumblesRecovered', 'value': 0.0},
  'fumblesRecoveredYards': {'name': 'fumblesRecoveredYards', 'value': 0.0},
  'fumblesTouchdowns': {'name': 'fumblesTouchdowns', 'value': 0.0},
  'gamesPlayed': {'name': 'gamesPlayed', 'value': 1.0},
  'defensiveFumblesTouchdowns': {'name': 'defensiveFumblesTouchdowns',
   'value': 0.0}},
 'defensive': {'name': 'defensive',
  'displayName': 'Defensive',
  'shortDisplayName': 'Defensive',
  'abbreviation': 'def',
  'summary': '',
  'assistTackles': {'name': 'assistTackles', 'value': 0.0},
  'avgInterceptionYards': {'name': 'avgInterceptionYards', 'value': 0.0},
  'avgSackYards': {'name': 'avgSackYards', 'value': 0.0},
  'avgStuffYards': {'name': 'avgStuffYards', 'value': 0.0},
  'blockedFieldGoalTouchdowns': {'name': 'blockedField

In [ ]:
pd.set_option("display.max_columns", None)
pd.json_normalize(game["competitors"], record_path="roster", record_prefix=".")

,.playerId,.period,.active,.starter,.forPlayerId,.valid,.didNotPlay,.athlete.$ref,.position.$ref,.statistics.general.name,.statistics.general.displayName,.statistics.general.shortDisplayName,.statistics.general.abbreviation,.statistics.general.summary,.statistics.general.fumblesForced.name,.statistics.general.fumblesForced.value,.statistics.general.fumblesRecovered.name,.statistics.general.fumblesRecovered.value,.statistics.general.fumblesRecoveredYards.name,.statistics.general.fumblesRecoveredYards.value,.statistics.general.fumblesTouchdowns.name,.statistics.general.fumblesTouchdowns.value,.statistics.general.gamesPlayed.name,.statistics.general.gamesPlayed.value,.statistics.general.defensiveFumblesTouchdowns.name,.statistics.general.defensiveFumblesTouchdowns.value,.statistics.defensive.name,.statistics.defensive.displayName,.statistics.defensive.shortDisplayName,.statistics.defensive.abbreviation,.statistics.defensive.summary,.statistics.defensive.assistTackles.name,.statistics.defensive.assistTackles.value,.statistics.defensive.avgInterceptionYards.name,.statistics.defensive.avgInterceptionYards.value,.statistics.defensive.avgSackYards.name,.statistics.defensive.avgSackYards.value,.statistics.defensive.avgStuffYards.name,.statistics.defensive.avgStuffYards.value,.statistics.defensive.blockedFieldGoalTouchdowns.name,.statistics.defensive.blockedFieldGoalTouchdowns.value,.statistics.defensive.blockedPuntTouchdowns.name,.statistics.defensive.blockedPuntTouchdowns.value,.statistics.defensive.defensiveTouchdowns.name,.statistics.defensive.defensiveTouchdowns.value,.statistics.defensive.hurries.name,.statistics.defensive.hurries.value,.statistics.defensive.kicksBlocked.name,.statistics.defensive.kicksBlocked.value,.statistics.defensive.longInterception.name,.statistics.defensive.longInterception.value,.statistics.defensive.miscTouchdowns.name,.statistics.defensive.miscTouchdowns.value,.statistics.defensive.passesBattedDown.name,.statistics.defensive.passesBattedDown.value,.statistics.defensive.passesDefended.name,.statistics.defensive.passesDefended.value,.statistics.defensive.QBHits.name,.statistics.defensive.QBHits.value,.statistics.defensive.twoPtReturns.name,.statistics.defensive.twoPtReturns.value,.statistics.defensive.sacks.name,.statistics.defensive.sacks.value,.statistics.defensive.sacksAssisted.name,.statistics.defensive.sacksAssisted.value,.statistics.defensive.sacksUnassisted.name,.statistics.defensive.sacksUnassisted.value,.statistics.defensive.sackYards.name,.statistics.defensive.sackYards.value,.statistics.defensive.safeties.name,.statistics.defensive.safeties.value,.statistics.defensive.soloTackles.name,.statistics.defensive.soloTackles.value,.statistics.defensive.stuffs.name,.statistics.defensive.stuffs.value,.statistics.defensive.stuffYards.name,.statistics.defensive.stuffYards.value,.statistics.defensive.tacklesForLoss.name,.statistics.defensive.tacklesForLoss.value,.statistics.defensive.tacklesYardsLost.name,.statistics.defensive.tacklesYardsLost.value,.statistics.defensive.teamGamesPlayed.name,.statistics.defensive.teamGamesPlayed.value,.statistics.defensive.totalTackles.name,.statistics.defensive.totalTackles.value,.statistics.defensive.yardsAllowed.name,.statistics.defensive.yardsAllowed.value,.statistics.defensive.pointsAllowed.name,.statistics.defensive.pointsAllowed.value,.statistics.defensive.onePtSafetiesMade.name,.statistics.defensive.onePtSafetiesMade.value,.statistics.defensiveInterceptions.name,.statistics.defensiveInterceptions.displayName,.statistics.defensiveInterceptions.shortDisplayName,.statistics.defensiveInterceptions.abbreviation,.statistics.defensiveInterceptions.summary,.statistics.defensiveInterceptions.interceptions.name,.statistics.defensiveInterceptions.interceptions.value,.statistics.defensiveInterceptions.interceptionTouchdowns.name,.statistics.defensiveInterceptions.interceptionTouchdowns.value,.statistics.defensiveInterceptions.interceptionYards.name,.statistics.defensiveIntercep

## Athlet


In [ ]:
url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2022/athletes/3915511?lang=en&region=us"
athlet = requests.get(url).json()

In [ ]:
def preprocess_athlet(athlet):
  athlet["position"]["parent"] = unwrap_reference(athlet["position"]["parent"])
  athlet["contracts"] = unwrap_reference(athlet["contracts"])

  # college info
  if ("collegeAthlete" in athlet and "college" in athlet):
    athlet["college"] = unwrap_reference(athlet["college"])
    athlet["collegeAthlete"] = unwrap_reference(athlet["collegeAthlete"])
    college_athlet = athlet["collegeAthlete"]
    if ("statisticslog" in college_athlet):
      college_athlet["statisticslog"] = unwrap_reference(college_athlet["statisticslog"])
      seasons = college_athlet["statisticslog"]["entries"]
      stats_seasons = []
      for season in seasons:
        stats = unwrap_reference(season["statistics"][0]["statistics"])["splits"]["categories"]
        stats_seasons.append({"season": season["season"]["$ref"]})
        stats_seasons[-1]["statistics"] = statistics_unwrap(stats)

    athlet["collegeAthlete"] = {
        "statistics": stats_seasons,
        "team": college_athlet["team"],
        "experience": college_athlet["experience"],
        "position": college_athlet["position"],
        "college": college_athlet["college"]
    }

  # overall stats
  if ("statistics" in athlet):
    athlet["statistics"] = unwrap_reference(athlet["statistics"])
    stats = athlet["statistics"]["splits"]["categories"]
    athlet["statistics"] = statistics_unwrap(stats)

  # games played
  if ("eventLog" in athlet):
    athlet["eventLog"] = requests.get(athlet["eventLog"]["$ref"]+"&limit=1000").json()
    if ("events" in athlet):
      athlet["eventLog"] = athlet["eventLog"]["events"]["items"]
      athlet["eventLog"] = [event["competition"] for event in athlet["eventLog"]]
    else:
      athlet["eventLog"] = []

  bad_cols = ["alternateIds", "links", "headshot"]
  for section in bad_cols:
    try:
      del athlet[section]
    except:
      pass

  json.dump(athlet, open(f"athlets/athlet_{athlet['id']}.json", "w"))
  return athlet

processed_athlet = preprocess_athlet(athlet.copy())

## Team

In [ ]:
def preprocess_team(team):
  # record
  if ("record" in team):
    team["record"] = unwrap_reference(team["record"])
    team["record"] = statistics_unwrap(team["record"])

  if ("statistics" in team):
    team["statistics"] = unwrap_reference(team["statistics"])["splits"]["categories"]
    team["statistics"] = statistics_unwrap(team["statistics"])

  if ("leaders" in team):
    team["leaders"] = unwrap_reference(team["leaders"])
    leaders = [] # or split on categories?
    for category in team["leaders"]["categories"]:
      for athlete in category["leaders"]:
        leaders.append({"category": category["name"], "athlete_id": extract_ids_from_url(athlete["athlete"]["$ref"])["athletes_id"]})
    team["leaders"] = leaders

  if ("againstTheSpreadRecords" in team):
    team["againstTheSpreadRecords"] = unwrap_reference(team["againstTheSpreadRecords"])
    stats = {}
    for stat in team["againstTheSpreadRecords"]:
      stat_name = stat["type"]["name"]
      stats[stat_name] = stat
    team["againstTheSpreadRecords"] = stats

  if ("events" in team):
    team["events"] = unwrap_reference(team["events"])

  if ("transactions" in team):
    team["transactions"] = unwrap_reference(team["transactions"])

  if ("coaches" in team):
    team["coaches"] = unwrap_reference(team["coaches"])

  if ("groups" in team):
    team["groups"] = unwrap_reference(team["groups"])

  if ("ranks" in team):
    team["ranks"] = unwrap_reference(team["ranks"])

  if ("attendance" in team):
    team["attendance"] = unwrap_reference(team["attendance"])
    team["attendance"] = statistics_unwrap(team["attendance"]["categories"])

  team.pop("logos", None)
  team.pop("links", None)
  team.pop("alternateIds", None)
  team.pop("injuries", None)
  team.pop("notes", None)
  team.pop("franchise", None)
  if "venue" in team:
    team["venue"].pop("images", None)
  json.dump(team, open(f"teams/team_{team['id']}.json", "w"))
  return team

url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2021/teams/25?lang=en&region=us"
team = requests.get(url).json()
team = preprocess_team(team)

# Items list getter

In [ ]:
if (not os.path.exists("athlets_list.csv")):
  athlet_list = [i["$ref"] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes")]
  df = pd.DataFrame(athlet_list)
  df.to_csv("athlets_list.csv")

In [ ]:
if (not os.path.exists("teams_list.csv")):
  games_list = [i["$ref"] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events")]
  games_list_df = pd.DataFrame(games_list)
  games_list_df.to_csv("teams_list.csv")

In [ ]:
if (not os.path.exists("events_list.csv")):
  games_list = []
  for season in range(2000, 2023):
    for week in range(40):
      for typ in range(5):
        try:
          games_list.extend([[season, typ, week, i["$ref"]] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events".format(season, typ, week))])
        except:
          continue
  games_list_df = pd.DataFrame(games_list, columns =["season", "type", "week", "link"])
  games_list_df.to_csv("events_list.csv")

In [ ]:
print("Num of games: ", games_list_df.shape[0])

Num of games:  335


# Crwler

Games

In [ ]:
!mkdir games

In [ ]:
from tqdm import tqdm
from datetime import datetime

for i in tqdm(range(games_list_df.shape[0])):
  url = games_list_df.loc[i, "link"]
  print(url)
  idg = extract_ids_from_url(url)["events_id"]
  if (os.path.exists(os.path.join("games", "game_"+str(idg)+".json"))):
    continue
  try:
    load_game(requests.get(url).json())
  except Exception as e:
    with open("error.txt", "a") as f:
      now = datetime.now()
      dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
      f.write(dt_string+ " -> ")
      f.write("---ERROR--- game id - "+str(idg)+" \n")
      f.write(e.__str__())
      f.write(str(e.__class__())+"\n")
      f.write(url)
      f.write(os.path.join("games", "game_"+str(idg)+".json"))
      f.write("\n \n")
  break

  0%|          | 0/335 [00:00<?, ?it/s]

http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401437964?lang=en&region=us


  0%|          | 0/335 [00:19<?, ?it/s]


# Convert JSON to table

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
!gdown 1Hl6BMVhEakIvEkqanYU2MoP0rp273mOU

Downloading...
From: https://drive.google.com/uc?id=1Hl6BMVhEakIvEkqanYU2MoP0rp273mOU
To: /content/games_0506_final.zip
100% 629M/629M [00:19<00:00, 31.5MB/s]


In [ ]:
!unzip -q /content/games_0506_final.zip
!mkdir newgames

In [ ]:
def process_games(json_name):
  if (os.path.exists("newgames/"+json_name)):
    return
  content = ""
  with open("games/"+json_name, "r") as f:
    content = json.loads(f.read())
  content.pop("$ref", None)
  content.pop("uid", None)
  content.pop("guid", None)
  content["date"] = pd.to_datetime(content["date"])
  content["season"] = content["date"].year
  if ("venue" in content):
    content["venue"].pop("$ref", None)
  for ct in content["competitors"]:
    ct.pop("$ref", None)
    ct.pop("uid", None)
    ct.pop("type", None)
    ct.pop("ranks", None)

    ct["team_id"] = ct["id"]
    del ct["id"]
    del ct["team"]
    if (not("score" in ct)):
      break
    ct["score"] = ct["score"]["value"]
    if ("leaders" in ct):
      try:
        ct["leaders"] = unwrap_reference(ct["leaders"])
      except:
        print("ERROR in leaders")
      if ("categories" in ct["leaders"]):
        leaders = {}
        for lead in ct["leaders"]["categories"]:
          if (len(lead["leaders"]) != 0):
            leaders[lead["name"]] = {"value": lead["leaders"][0]["value"],
                                     "ahlete": extract_ids_from_url(
                                      lead["leaders"][0]["athlete"]["$ref"])
                                     ["athletes_id"] # exactly 0 only!
            }
        ct["leaders"] = leaders
    ct.pop("linescores", None)
    ct.pop("nextCompetition", None)
    ct.pop("previousCompetition", None)
    ct.pop("order", None)
    for rc in ct["record"]:
      rc.pop("$ref", None)
      rc.pop("id", None)
      rc.pop("name", None)
      rc.pop("displayValue", None)
      rc.pop("abbreviation", None)
      rc.pop("shortDisplayName", None)
      rc.pop("description", None)
      rc.pop("displayValue", None)
      rc.pop("summary", None)
      rc.update({st["type"]: st["value"] for st in rc["stats"]})
      rc.pop("stats", None)
    ct["record"] = {rc["type"]: rc for rc in ct["record"]}
    if ("statistics" in ct):
      ct["statistics"] = statistics_unwrap(ct["statistics"])
      for stat in ct["statistics"]:
        ct["statistics"][stat].pop("athletes", None)
        ct["statistics"][stat].pop("name", None)
        ct["statistics"][stat].pop("displayName", None)
        ct["statistics"][stat].pop("shortDisplayName", None)
        ct["statistics"][stat].pop("abbreviation", None)
        ct["statistics"][stat].pop("summary", None)
        ct["statistics"][stat] = {stat_field: ct["statistics"][stat][stat_field]["value"] for stat_field in ct["statistics"][stat]}
      if ("roster" in ct):
        for rost in ct["roster"]:
          rost.pop("athlete", None)
          rost["position"] = extract_ids_from_url(rost["position"]["$ref"])["positions_id"]
          rost.pop("displayName", None)
          rost.pop("valid", None)
          rost.pop("forPlayerId", None)
          if ("statistics" in rost):
            for stat in rost["statistics"]:
              rost["statistics"][stat].pop("athletes", None)
              rost["statistics"][stat].pop("name", None)
              rost["statistics"][stat].pop("displayName", None)
              rost["statistics"][stat].pop("shortDisplayName", None)
              rost["statistics"][stat].pop("abbreviation", None)
              rost["statistics"][stat].pop("summary", None)
              rost["statistics"][stat] = {stat_field: rost["statistics"][stat][stat_field]["value"] for stat_field in rost["statistics"][stat]}
  content.pop("notes", None)
  content.pop("odds", None)
  content.pop("predictor", None)
  content.pop("probabilities", None)
  content.pop("powerIndexes", None)
  content.pop("format", None)
  content.pop("officials", None)
  content.pop("details", None)
  content.pop("drives", None)
  content.pop("tickets", None)
  content.pop("leaders", None)
  content.pop("attendance", None)

  json.dump(content, open("newgames/"+json_name, "w"), default=str)

for json_name in tqdm(os.listdir("games")):
  process_games(json_name)

 20%|██        | 1605/7968 [15:39<48:56,  2.17it/s]

ERROR


 20%|██        | 1606/7968 [15:40<58:15,  1.82it/s]

ERROR


 20%|██        | 1613/7968 [15:49<3:14:06,  1.83s/it]

ERROR


100%|██████████| 7968/7968 [1:17:10<00:00,  1.72it/s]


In [ ]:
!zip -r updated_games.zip newgames/

In [ ]:
# Create & upload a text file.
uploaded = drive.CreateFile({'title': "updated_games.zip"})
uploaded.SetContentFile("updated_games.zip")
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

KeyboardInterrupt: ignored

players

In [ ]:
# pd.DataFrame().to_csv("players_games.csv")

# def tocsv_transformer(json_name, header=True):
#   content = ""
#   with open("newgames/"+json_name, "r") as f:
#     content = json.loads(f.read())
#   roster_df = pd.DataFrame()
#   for team in content["competitors"]:
#     if "roster" in team:
#       roster_df = pd.json_normalize(team["roster"])
#       roster_df["season"] = content["season"]
#       roster_df["play_id"] = content["id"]
#       if ("team_id" in team):
#         roster_df["team_id"] = team["team_id"]
#       else:
#         return None
#     else:
#       return None
#     roster_df.to_csv("players_games.csv", mode='a')

# with ThreadPoolExecutor(max_workers=16) as pool:
#   pool.map(tocsv_transformer, os.listdir("newgames"))

# df = pd.read_csv("players_games.csv", on_bad_lines='skip', low_memory=False)
# df_players = df.reset_index()
# df_players.columns = df_players.loc[0]
# df_players = df_players.iloc[1:, 1:]
# df_players.to_csv("dataset_game_players.csv", chunksize=1000000)

In [ ]:
# # Create & upload a text file.
# uploaded = drive.CreateFile({'title': "dataset_game_players.csv"})
# uploaded.SetContentFile("dataset_game_players.csv")
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1HbZ3FUWkkhDsDjTf1aP8-ZqwXrNIyQaP


# Prepared json teams to csv

In [ ]:
a = set()
for json_name in tqdm(os.listdir("newgames/")[5000:7900]):
  with open("newgames/"+json_name, "r") as f:
    content = json.loads(f.read())
  team_df = pd.DataFrame()
  a.update([i for i in pd.json_normalize(content).columns if not i in ["competitors", "id", "season"] and not i.startswith("venue")])
  for team in content["competitors"]:
    team_df = pd.concat([team_df, pd.json_normalize(team)])
  team_df["season_id"] = content["season"]
  team_df["game_id"] = content["id"]
  if ("roster" in team_df.columns):
    team_df = team_df.drop("roster", axis=1)
  team
  a.update(team_df.columns)
dataset_columns = sorted(list(a))

In [ ]:
dataset_columns

In [ ]:
pd.DataFrame().to_csv("dataset_teams_games.csv")
header = True
def tocsv_transformer_teams(json_name, header):
  content = ""
  with open("newgames/"+json_name, "r") as f:
    content = json.loads(f.read())

  team_df = pd.DataFrame(columns = dataset_columns)
  for team in content["competitors"]:
    data_piece = pd.json_normalize(team)
    new_row = {i: None for i in dataset_columns}
    for i in data_piece.columns:
      new_row[i] = [data_piece.loc[0, i]]
    team_df = pd.concat([team_df, pd.DataFrame(new_row)])

  team_df["season_id"] = content["season"]
  team_df["game_id"] = content["id"]
  team_df["date"] = content["date"]
  if ("roster" in team_df.columns):
    team_df = team_df.drop("roster", axis=1)
  team_df.to_csv("dataset_teams_games.csv", header=header, mode='a')

for json_name in tqdm(os.listdir("newgames/")):
  tocsv_transformer_teams(json_name, header)
  header=False

df_teams = pd.read_csv("dataset_teams_games.csv", on_bad_lines="skip")
df_teams = df_teams.reset_index()
df_teams.columns = df_teams.iloc[0]
df_teams = df_teams.iloc[1:, 1:]
df_teams.to_csv("dataset_teams_games.csv")

In [ ]:
df_teams[["team_id", "season_id", "game_id", "score", "winner"]]

,team_id,season_id,play_id,score,winner
1,9,2006,260101009,23.0,True
2,26,2006,260101009,17.0,False
3,15,2021,401326353,0.0,False
4,2,2021,401326353,35.0,True
5,18,2005,251030018,6.0,False
...,...,...,...,...,...
15928,25,2002,221221022,17.0,True
15929,21,2013,330815021,14.0,True
15930,29,2013,330815021,9.0,False
15931,20,2003,231012020,30.0,True


In [ ]:
df_teams.to_csv("dataset_teams_games.csv")

In [ ]:
# Create & upload a text file.
uploaded = drive.CreateFile({'title': "dataset_teams_games.csv"})
uploaded.SetContentFile("dataset_teams_games.csv")
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1IBs5q9zzhM7lzMtYxE13INrKVzGHAovJ
